In [1]:
import csv

In [2]:
import json

In [3]:
import pandas

In [4]:
import pymysql

In [5]:
import emoji

In [6]:
json_content = []
with open('data_portfolio_21.csv', encoding='utf8', newline='') as f:
    reader = csv.DictReader(f)
    for row in reader:
        json_content.append(row)

# Write json to file
with open('riddit.json', 'w', encoding='utf8') as f:
    json.dump(json_content, f, indent=4)

In [7]:
reddit_data = json.load(open('riddit.json'))

In [8]:
password = 'password'
# Connect to the database
connection = pymysql.connect(host='csmysql.cs.cf.ac.uk',
                             user='c2090580',
                             password=password,
                             db='c2090580_assessment_1',
                             charset='utf8mb4')

In [9]:
cursor = connection.cursor()

In [10]:
with connection.cursor() as cursor:
    sql = """drop table if exists posts_info;"""
    cursor.execute(sql)
    connection.commit()
with connection.cursor() as cursor:
    sql = """drop table if exists user_info;"""
    cursor.execute(sql)
    connection.commit()
with connection.cursor() as cursor:
    sql = """drop table if exists subreddit_info;"""
    cursor.execute(sql)
    connection.commit()


In [11]:
#USER INFORMATION
# user info into a table
author_map = {}
for author_names in reddit_data:
    author_name = author_names['author']
    if not author_name in author_map:
        author_map[author_name] = len(author_map)

In [12]:
#except duplicate user
duplicate_author_set = set()
user_info_list = []
for user_info in reddit_data:
    user_name = user_info['author']
    user_id = author_map[user_name]
    user_num_posts = user_info['user_num_posts']
    user_registered_at = user_info['user_registered_at']
    user_upvote_ratio = user_info['user_upvote_ratio']
    if user_id in duplicate_author_set:
        continue
    this_list = [user_id,user_name,user_num_posts,user_registered_at,user_upvote_ratio]
    user_info_list.append(this_list)
    duplicate_author_set.add(user_id)


In [13]:
# create user info table
with connection.cursor() as cursor:
    sql = """
        create table user_info(
        user_id int unsigned not null,
        user_name nvarchar(500),
        user_num_posts int,
        user_registered_at date,
        user_upvote_ratio double,
        primary key (user_id)
);

    """
    cursor.execute(sql)
    connection.commit()


In [14]:
# insert user info into table
with connection.cursor() as cursor:
    sql = """
    insert into user_info (user_id,user_name,user_num_posts,user_registered_at,user_upvote_ratio) values (%s,%s,%s,%s,%s);

    """
    cursor.executemany(sql, user_info_list)
    connection.commit()

In [15]:
#SUB REDDIT INFOMATION
duplicate_subreddit_set = set()
sub_reddit_map = {}

for subreddit_names in reddit_data:
    subreddit_name = subreddit_names['subreddit']
    if not subreddit_name in sub_reddit_map:
        sub_reddit_map[subreddit_name] = len(sub_reddit_map)


subreddit_info_list = []
for subreddit_info in reddit_data:
    subreddit_name = subreddit_info['subreddit']
    subreddit_id = sub_reddit_map[subreddit_name]
    subreddit_created_at = subreddit_info['subr_created_at']
    subreddit_description = subreddit_info['subr_description']
    subreddit_faved_by = subreddit_info['subr_faved_by']
    subreddit_numb_members = subreddit_info['subr_numb_members']
    subreddit_numb_posts = subreddit_info['subr_numb_posts']
    if subreddit_id in duplicate_subreddit_set:
        continue
    this_list = [subreddit_id,subreddit_name,subreddit_created_at,subreddit_description,subreddit_faved_by,subreddit_numb_members,subreddit_numb_posts]
    subreddit_info_list.append(this_list)
    duplicate_subreddit_set.add(subreddit_id)

In [16]:
#create subreddit table and insert data into it
#
with connection.cursor() as cursor:
    sql = """
        create table subreddit_info(
        subreddit_id int unsigned not null,
        subreddit_name nvarchar(500),
        subreddit_created_at date,
        subreddit_description nvarchar(500),
        subreddit_faved_by longtext,
        subreddit_numb_members int,
        subreddit_numb_posts int,
        primary key (subreddit_id)
    );
    
    """
    cursor.execute(sql)
    connection.commit()

with connection.cursor() as cursor:
    sql = """ALTER TABLE subreddit_info CONVERT TO CHARACTER SET utf8mb4;"""
    cursor.execute(sql)
    connection.commit()

with connection.cursor() as cursor:
    sql = """
    
    insert into subreddit_info (subreddit_id,subreddit_name,subreddit_created_at,subreddit_description,subreddit_faved_by,subreddit_numb_members,subreddit_numb_posts) values (%s,%s,%s,%s,%s,%s,%s)
    """
    cursor.executemany(sql,subreddit_info_list)
    connection.commit()

In [17]:
#POSTS
#this block is about post infomation

post_info_list = []
for posts_info in reddit_data:

    post_id = len(post_info_list)
    post_title = posts_info['title']

    user_name = posts_info['author']
    user_id = author_map[user_name]

    subreddit_name = posts_info['subreddit']
    subreddit_id = sub_reddit_map[subreddit_name]

    posted_at = posts_info['posted_at']
    post_num_comments = posts_info['num_comments']
    post_score = posts_info['score']
    post_total_awards_received = posts_info['total_awards_received']
    post_upvote_ratio = posts_info['upvote_ratio']
    selftext = posts_info['selftext']

    this_list = [post_id,post_title,user_id,subreddit_id,posted_at,post_num_comments,post_score,post_total_awards_received,post_upvote_ratio,selftext]
    post_info_list.append(this_list)
    duplicate_author_set.add(user_id)
    duplicate_subreddit_set.add(subreddit_id)

In [18]:
with connection.cursor() as cursor:
    sql = """
        create table posts_info(
        post_id int unsigned not null,
        post_title nvarchar(500),
        post_author_id int unsigned not null,
        post_subreddit_id int unsigned not null,
        posted_at datetime,
        post_num_comments int,
        post_score int,
        post_total_awards_received int,
        post_upvote_ratio float,
        selftext longtext,
        constraint `fk_author` foreign key (post_author_id) references user_info(user_id) on delete restrict on update cascade,
        constraint `fk_subreddit` foreign key (post_subreddit_id) references subreddit_info(subreddit_id) on delete restrict on update cascade
    );

    """
    cursor.execute(sql)
    connection.commit()



with connection.cursor() as cursor:
    sql = """ALTER TABLE posts_info CONVERT TO CHARACTER SET utf8mb4;"""
    cursor.execute(sql)
    connection.commit()

with connection.cursor() as cursor:
    sql = """
    
    insert into posts_info(post_id,post_title,post_author_id,post_subreddit_id,posted_at,post_num_comments,post_score,post_total_awards_received,post_upvote_ratio,selftext) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);
    
    """
    cursor.executemany(sql,post_info_list)
    connection.commit()

In [19]:
# sub_favourite_users
df = pandas.read_csv('data_portfolio_21.csv').fillna('NULL')
subr_faved_users = df['subr_faved_by'].values.tolist()
subr_faved_usernames = []
for i in range(len(subr_faved_users)):
    username_list = subr_faved_users[i][1:-1].split(",")
    if len(username_list) == 0:
        print(" ", subr_faved_users[i])
    for usernames in username_list:
        name = usernames.strip()[1:-1]
        subr_faved_usernames.append((i, name))

In [20]:
with connection.cursor() as cursor:
    sql = """drop table if exists favorite_user_info;"""
    cursor.execute(sql)
    connection.commit()

with connection.cursor() as cursor:
    sql = """
        create table favorite_user_info(
        fav_user_id int auto_increment,
        reddit_id int unsigned not null,
        fav_user_name varchar(255),
        primary key (fav_user_id)
        
    );
    """
    cursor.execute(sql)
    connection.commit()

with connection.cursor() as cursor:
    sql = """ALTER TABLE favorite_user_info CONVERT TO CHARACTER SET utf8mb4;"""
    cursor.execute(sql)
    connection.commit()

with connection.cursor() as cursor:
    sql = """

    insert into favorite_user_info(reddit_id, fav_user_name) values (%s, %s);

    """
    cursor.executemany(sql, subr_faved_usernames)
    connection.commit()